# exploración de datos

In [3]:
# ================================================================
# 02_build_product_level.ipynb
# - Limpieza de datos a nivel fila (producto-semana)
# - Construcción de dataset agregado a nivel producto
# - Feature engineering "macro" (producto / producto-season)
# ================================================================

import pandas as pd
import numpy as np
from pathlib import Path

In [4]:
# ================================================================
# 1. Cargar datos
# ================================================================
# Ajusta la ruta al CSV/parquet real
DATA_PATH = Path("../data/train.csv")

df = pd.read_csv(DATA_PATH, delimiter=";")

In [5]:
# ================================================================
# 2. Limpieza de tipos de dato
# ================================================================
# Estrategia:
# - Fechas/semana en formato manejable
# - Categorías como 'category', 'fabric', etc. se mantienen como 'object'
#   (más tarde se codificarán en 03_feature_engineering)


# Si phase_in / phase_out vienen como fechas, las convertimos
date_cols = ["phase_in", "phase_out"]
for c in date_cols:
    if c in df.columns:
        # errors='coerce' convierte valores raros a NaT (los podremos filtrar)
        df[c] = pd.to_datetime(df[c], errors="coerce")


/tmp/ipykernel_322338/2752578634.py:15: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[c] = pd.to_datetime(df[c], errors="coerce")


In [6]:
# ================================================================
# 3. Limpieza básica de registros
# ================================================================
# Estrategia:
# - Quitar filas completamente vacías en las columnas clave
# - Eliminar registros con IDs nulos
# - Quitar filas con Production negativa o absurda
# - Quitar weekly_sales / weekly_demand negativas (no tienen sentido)

# 4.1. Filtrar filas con ID nulo
df = df[~df["ID"].isna()].copy()

# 4.2. Filtrar valores negativos raros
for col in ["weekly_sales", "weekly_demand", "Production"]:
    if col in df.columns:
        df = df[df[col].isna() | (df[col] >= 0)]

# 4.3. Opcional: filtro muy extremo de outliers de Production
# (por ejemplo, eliminar el 0.5% superior si está totalmente fuera de rango)
if "Production" in df.columns:
    upper = df["Production"].quantile(0.995)
    df = df[df["Production"] <= upper]


In [7]:
# ================================================================
# 5. Construcción de identificador Season-Year
# ================================================================
# Estrategia:
# - Tenemos 4 seasons al año durante 4 años.
# - Es útil combinar year + id_season para crear una "temporada única"
#   y poder agrupar por product-season (para productos revendidos).

if {"id_season","year"}.issubset(df.columns):
    df["season_year"] = df["year"].astype(str) + "_" + df["id_season"].astype(str)
else:
    df["season_year"] = np.nan


In [8]:
# ================================================================
# 8. Agregación a nivel producto-season
#    (cada fila: un producto en una temporada concreta)
# ================================================================
# Estrategia:
# - Como hay productos revendidos en varias temporadas, es mejor crear
#   primero un dataset a nivel (ID, season_year) en vez de sólo ID.
# - Luego, si quieres, se puede volver a agregar a puro nivel producto.

group_cols = ["ID", "season_year"]

# Diccionario de agregaciones: escogemos métricas que tengan sentido
agg_num = {
    "weekly_sales": ["sum", "mean", "max"],
    "weekly_demand": ["sum", "mean", "max"],
    "Production": "max",                 # producción total asignada a ese producto en esa temporada
    "num_stores": "max",
    "num_sizes": "max",
    "life_cycle_length": "max",
    "price": ["mean", "max", "min"]
}

# Columnas categóricas que deberían ser constantes dentro de un product-season
cat_cols = [
    "id_season","aggregated_family","family","category","fabric",
    "color_name","length_type","silhouette_type",
    "waist_type","sleeve_length_type","heel_shape_type",
    "toecap_type","woven_structure","knit_structure","print_type",
    "archetype","moment","ocassion"
]

agg_cat = {c: "first" for c in cat_cols if c in df.columns}

# Fechas
for c in date_cols:
    if c in df.columns:
        agg_cat[c] = "first"

product_season = (
    df
    .groupby(group_cols, as_index=False)
    .agg({**agg_num, **agg_cat})
)

# Aplastar columnas multi-índice del groupby
product_season.columns = [
    "_".join([c for c in col if c])  # une niveles no vacíos
    if isinstance(col, tuple) else col
    for col in product_season.columns
]

print(product_season.shape)
product_season.head()


(10973, 34)


,ID,season_year,weekly_sales_sum,weekly_sales_mean,weekly_sales_max,weekly_demand_sum,weekly_demand_mean,weekly_demand_max,Production_max,num_stores_max,...,sleeve_length_type_first,heel_shape_type_first,toecap_type_first,woven_structure_first,knit_structure_first,print_type_first,archetype_first,moment_first,phase_in_first,phase_out_first
0,1,2023_86,802,66.833333,135,806,67.166667,135,4556,152,...,Fine Strap,NaN,NaN,Light,None,Sin Estampado,None,TIME OFF,2023-02-01,2023-03-26
1,2,2023_88,217,108.500000,127,228,114.000000,138,12267,121,...,Short,NaN,NaN,Light,None,Sin Estampado,Casual Chic,CASUAL WORK,NaT,2024-03-10
2,2,2024_88,1968,196.800000,258,2038,203.800000,259,12267,121,...,Short,NaN,NaN,Light,None,Sin Estampado,Casual Chic,CASUAL WORK,NaT,2024-03-10
3,3,2024_89,55472,3081.777778,4577,63791,3543.944444,5441,137780,1134,...,None,NaN,NaN,None,None,Sin Estampado,Authentic,TIME OFF,NaT,2024-11-17
4,4,2024_89,10062,1257.750000,2326,11004,1375.500000,2672,33001,1090,...,Sleeveless,NaN,NaN,Light,None,Rayas,None,FORMAL WORK,NaT,2024-10-20


In [9]:
# ================================================================
# 9. Feature engineering a nivel producto-season
# ================================================================
# Estrategia general:
# - Queremos capturar:
#   * intensidad de ventas/demanda
#   * eficiencia de producción (sell-through)
#   * complejidad del producto (nº de tiendas, tallas, plus sizes)
#   * duración real del ciclo vs planificado
#   * señales temporales (año, season)
# - Aquí dejamos features "de negocio" y más adelante (03_feature_engineering)
#   haremos encoding avanzado, escalado, interacciones, etc.

ps = product_season.copy()

# --- 9.1 KPI de ciclo de vida ---
# Ventas totales y demanda total ya están agregadas
ps["sell_through_ratio"] = ps["weekly_sales_sum"] / ps["Production_max"]
ps["demand_coverage_ratio"] = ps["Production_max"] / ps["weekly_demand_sum"]

# Limitamos a 0-2 para evitar valores extremos raros (clipping)
ps["sell_through_ratio"] = ps["sell_through_ratio"].clip(0, 2)
ps["demand_coverage_ratio"] = ps["demand_coverage_ratio"].clip(0, 2)

# --- 9.2 Intensidad de ventas semanal ---
ps["avg_sales_per_week_planned"] = ps["weekly_sales_sum"] / ps["life_cycle_length_max"]
ps["avg_demand_per_week_planned"] = ps["weekly_demand_sum"] / ps["life_cycle_length_max"]

# --- 9.3 Complejidad del surtido ---
ps["stores_per_size"] = ps["num_stores_max"] / ps["num_sizes_max"]

# --- 9.4 Señales temporales ---
# Extraemos year de 'season_year' si hace falta
if "season_year" in ps.columns:
    # season_year = "2021_1" → year=2021, season_id=1
    ps["year_from_season"] = ps["season_year"].str.split("_").str[0].astype(int)
    ps["season_id"] = ps["season_year"].str.split("_").str[1].astype(int)

# --- 9.5 Flags simples de tipo de producto (ejemplo muy básico) ---
# Estas features son útiles porque con ~9.8k productos en 4 años * 4 seasons
# necesitamos dar al modelo una forma de agrupar comportamientos parecidos.

if "aggregated_family_first" in ps.columns:
    ps["is_outerwear"] = ps["aggregated_family_first"].str.contains(
        "Coat|Jacket|Outer", case=False, na=False
    ).astype(int)

if "moment_first" in ps.columns:
    ps["is_formal"] = ps["moment_first"].str.contains(
        "Formal", case=False, na=False
    ).astype(int)
    ps["is_casual"] = ps["moment_first"].str.contains(
        "Casual", case=False, na=False
    ).astype(int)


In [10]:
# ================================================================
# 11. Seleccionar columnas finales y guardar
# ================================================================
# Estrategia:
# - Dejamos:
#   * ID y season_year como identificadores
#   * Variables objetivo: 'Production_max' (target de producción)
#   * Features agregadas y de negocio
# - El encoding de categóricas y preparación para XGBoost se hará en 03_feature_engineering

id_cols = ["ID", "season_year"]

target_col = "Production_max"   # target a predecir (unidades de producción)

feature_cols = [
    # KPI de demanda/ventas
    "weekly_sales_sum",
    "weekly_sales_mean",
    "weekly_sales_max",
    "weekly_demand_sum",
    "weekly_demand_mean",
    "weekly_demand_max",
    # Plan de vida y surtido
    "life_cycle_length_max",
    "num_stores_max",
    "num_sizes_max",
    "stores_per_size",
    # Precio
    "price_mean",
    "price_max",
    "price_min",
    # Ratios de eficiencia
    "sell_through_ratio",
    "demand_coverage_ratio",
    "avg_sales_per_week_planned",
    "avg_demand_per_week_planned",
    # Color numérico
    "color_r_mean",
    "color_g_mean",
    "color_b_mean",
    # Temporales
    "year_from_season",
    "season_id",
    # Históricos y flags
    "n_seasons_per_id",
    "season_order_for_id",
    "cumulative_production_before",
    "cumulative_demand_before",
    "is_outerwear",
    "is_formal",
    "is_casual",
]

# añadimos algunas categóricas "crudas" que luego se codificarán
raw_cat_cols = [
    "aggregated_family_first",
    "family_first",
    "category_first",
    "fabric_first",
    "color_name_first",
    "length_type_first",
    "silhouette_type_first",
    "waist_type_first",
    "sleeve_length_type_first",
    "archetype_first",
    "moment_first",
    "ocassion_first",
]

cols_existing = id_cols + [target_col] + [c for c in feature_cols + raw_cat_cols if c in ps.columns]

dataset_product_level = ps[cols_existing].copy()

print(dataset_product_level.shape)
dataset_product_level.head()


(10973, 36)


,ID,season_year,Production_max,weekly_sales_sum,weekly_sales_mean,weekly_sales_max,weekly_demand_sum,weekly_demand_mean,weekly_demand_max,life_cycle_length_max,...,family_first,category_first,fabric_first,color_name_first,length_type_first,silhouette_type_first,waist_type_first,sleeve_length_type_first,archetype_first,moment_first
0,1,2023_86,4556,802,66.833333,135,806,67.166667,135,12,...,Dresses,"Dresses, jumpsuits and Complete set",WOVEN,AMARILLO,Long,Evase,None,Fine Strap,None,TIME OFF
1,2,2023_88,12267,217,108.500000,127,228,114.000000,138,12,...,Shirt,Tops,WOVEN,VERDE PASTEL,Crop,Straight,None,Short,Casual Chic,CASUAL WORK
2,2,2024_88,12267,1968,196.800000,258,2038,203.800000,259,12,...,Shirt,Tops,WOVEN,VERDE PASTEL,Crop,Straight,None,Short,Casual Chic,CASUAL WORK
3,3,2024_89,137780,55472,3081.777778,4577,63791,3543.944444,5441,18,...,Jeans,Bottoms,JEANS,ARENA,Ankle brace,Culotte,High Waist,None,Authentic,TIME OFF
4,4,2024_89,33001,10062,1257.750000,2326,11004,1375.500000,2672,8,...,Shirt,Tops,WOVEN,CELESTE,Standard,Regular,None,Sleeveless,None,FORMAL WORK


In [11]:
# Guardar el dataset para usar en 03_feature_engineering
OUTPUT_PATH = Path("../data/processed/processed_data.csv")
OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)

df.to_csv("../data/processed_data.csv", index=False)

print(f"Dataset a nivel producto-season guardado en: {OUTPUT_PATH}")


Dataset a nivel producto-season guardado en: ../data/processed/processed_data.csv
